In [6]:
import numpy as np
import pandas as pd
import time
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RANSACRegressor, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import warnings
warnings.filterwarnings('ignore')

In [7]:
california_data = fetch_california_housing()
X = california_data.data
y = california_data.target

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
alpha_values = [0.5, 1, 10]
max_trials_values = [50, 100, 150]
degree_values = [1, 2, 3]

In [10]:
models = {
    'LinearRegression': LinearRegression(fit_intercept=False),
    'RANSACRegressor': [RANSACRegressor(base_estimator=LinearRegression(fit_intercept=False), max_trials=max_trial) for max_trial in max_trials_values],
    'Ridge': [Ridge(alpha=alpha, fit_intercept=False) for alpha in alpha_values],
    'Lasso': [Lasso(alpha=alpha, fit_intercept=False) for alpha in alpha_values],
    'ElasticNet': [ElasticNet(alpha=alpha, l1_ratio=0.5, fit_intercept=False) for alpha in alpha_values],
    'PolynomialRegression': [make_pipeline(PolynomialFeatures(degree=degree), LinearRegression(fit_intercept=False)) for degree in degree_values]
}

results = []

for name, model_list in models.items():
    if isinstance(model_list, list):
        for model in model_list:
            start_time = time.time()
            model.fit(X_train, y_train)
            train_time = time.time() - start_time
            y_train_pred = model.predict(X_train)
            y_test_pred = model.predict(X_test)

            if name == 'RANSACRegressor':
                param_info = f"max_trials={model.max_trials}"
            elif name == 'PolynomialRegression':
                param_info = f"degree={model.steps[0][1].degree}"
            else:
                param_info = f"alpha={model.alpha}"

            results.append({
                'Model': f'{name}_{param_info}',
                'Parameters': model.get_params(),
                'Train MSE': mean_squared_error(y_train, y_train_pred),
                'Test MSE': mean_squared_error(y_test, y_test_pred),
                'Train R2': r2_score(y_train, y_train_pred),
                'Test R2': r2_score(y_test, y_test_pred),
                'Training Time': train_time
            })
    else:
        start_time = time.time()
        model_list.fit(X_train, y_train)
        train_time = time.time() - start_time
        y_train_pred = model_list.predict(X_train)
        y_test_pred = model_list.predict(X_test)
        results.append({
            'Model': name,
            'Parameters': model_list.get_params(),
            'Train MSE': mean_squared_error(y_train, y_train_pred),
            'Test MSE': mean_squared_error(y_test, y_test_pred),
            'Train R2': r2_score(y_train, y_train_pred),
            'Test R2': r2_score(y_test, y_test_pred),
            'Training Time': train_time
        })

In [11]:
results_df = pd.DataFrame(results)
print(results_df[[ 'Model', 'Train MSE', 'Test MSE', 'Train R2', 'Test R2', 'Training Time']])

                             Model  Train MSE   Test MSE  Train R2    Test R2  \
0                 LinearRegression   0.599036   0.629447  0.551881   0.519656   
1    RANSACRegressor_max_trials=50   9.379054   0.974782 -6.016163   0.256124   
2   RANSACRegressor_max_trials=100   2.009549   0.742053 -0.503278   0.433724   
3   RANSACRegressor_max_trials=150   3.067776   0.809778 -1.294903   0.382042   
4                  Ridge_alpha=0.5   0.599036   0.629402  0.551881   0.519690   
5                    Ridge_alpha=1   0.599036   0.629357  0.551881   0.519725   
6                   Ridge_alpha=10   0.599050   0.628588  0.551870   0.520311   
7                  Lasso_alpha=0.5   0.718703   0.720301  0.462362   0.450324   
8                    Lasso_alpha=1   0.924472   0.914311  0.308432   0.302270   
9                   Lasso_alpha=10   1.349713   1.319689 -0.009676  -0.007082   
10            ElasticNet_alpha=0.5   0.679509   0.685145  0.491682   0.477152   
11              ElasticNet_a